# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 15 2022 4:39PM,253238,10,CTF0011545,"Citieffe, Inc.",Released
1,Dec 15 2022 4:36PM,253236,10,CTF0011546,"Citieffe, Inc.",Released
2,Dec 15 2022 4:34PM,253235,19,60018271,"GUSA Granules USA, Inc.",Released
3,Dec 15 2022 4:34PM,253235,19,60018264,"GUSA Granules USA, Inc.",Released
4,Dec 15 2022 4:17PM,253230,10,0086251085,ISDIN Corporation,Released
5,Dec 15 2022 4:17PM,253230,10,0086251150,ISDIN Corporation,Released
6,Dec 15 2022 4:17PM,253230,10,0086251153,ISDIN Corporation,Released
7,Dec 15 2022 4:17PM,253230,10,0086251162,ISDIN Corporation,Released
8,Dec 15 2022 4:17PM,253230,10,0086251163,ISDIN Corporation,Released
9,Dec 15 2022 4:17PM,253230,10,0086251183,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,253230,Released,10
24,253232,Released,1
25,253235,Released,2
26,253236,Released,1
27,253238,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253230,NaN,NaN,10.0
253232,NaN,NaN,1.0
253235,NaN,NaN,2.0
253236,NaN,NaN,1.0
253238,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253179,0.0,0.0,1.0
253181,0.0,0.0,1.0
253184,6.0,4.0,5.0
253186,0.0,0.0,2.0
253195,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253179,0,0,1
253181,0,0,1
253184,6,4,5
253186,0,0,2
253195,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253179,0,0,1
1,253181,0,0,1
2,253184,6,4,5
3,253186,0,0,2
4,253195,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253179,,,1
1,253181,,,1
2,253184,6,4,5
3,253186,,,2
4,253195,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 15 2022 4:39PM,253238,10,"Citieffe, Inc."
1,Dec 15 2022 4:36PM,253236,10,"Citieffe, Inc."
2,Dec 15 2022 4:34PM,253235,19,"GUSA Granules USA, Inc."
4,Dec 15 2022 4:17PM,253230,10,ISDIN Corporation
14,Dec 15 2022 4:17PM,253232,22,"NBTY Global, Inc."
15,Dec 15 2022 4:16PM,253229,10,ISDIN Corporation
21,Dec 15 2022 4:09PM,253228,10,ISDIN Corporation
23,Dec 15 2022 4:03PM,253219,10,ISDIN Corporation
35,Dec 15 2022 3:55PM,253227,18,Innova Softgel LLC
36,Dec 15 2022 3:52PM,253226,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 15 2022 4:39PM,253238,10,"Citieffe, Inc.",,,1
1,Dec 15 2022 4:36PM,253236,10,"Citieffe, Inc.",,,1
2,Dec 15 2022 4:34PM,253235,19,"GUSA Granules USA, Inc.",,,2
3,Dec 15 2022 4:17PM,253230,10,ISDIN Corporation,,,10
4,Dec 15 2022 4:17PM,253232,22,"NBTY Global, Inc.",,,1
5,Dec 15 2022 4:16PM,253229,10,ISDIN Corporation,,,6
6,Dec 15 2022 4:09PM,253228,10,ISDIN Corporation,,,2
7,Dec 15 2022 4:03PM,253219,10,ISDIN Corporation,,,12
8,Dec 15 2022 3:55PM,253227,18,Innova Softgel LLC,,,1
9,Dec 15 2022 3:52PM,253226,10,ISDIN Corporation,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 4:39PM,253238,10,"Citieffe, Inc.",1,,
1,Dec 15 2022 4:36PM,253236,10,"Citieffe, Inc.",1,,
2,Dec 15 2022 4:34PM,253235,19,"GUSA Granules USA, Inc.",2,,
3,Dec 15 2022 4:17PM,253230,10,ISDIN Corporation,10,,
4,Dec 15 2022 4:17PM,253232,22,"NBTY Global, Inc.",1,,
5,Dec 15 2022 4:16PM,253229,10,ISDIN Corporation,6,,
6,Dec 15 2022 4:09PM,253228,10,ISDIN Corporation,2,,
7,Dec 15 2022 4:03PM,253219,10,ISDIN Corporation,12,,
8,Dec 15 2022 3:55PM,253227,18,Innova Softgel LLC,1,,
9,Dec 15 2022 3:52PM,253226,10,ISDIN Corporation,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 4:39PM,253238,10,"Citieffe, Inc.",1,,
1,Dec 15 2022 4:36PM,253236,10,"Citieffe, Inc.",1,,
2,Dec 15 2022 4:34PM,253235,19,"GUSA Granules USA, Inc.",2,,
3,Dec 15 2022 4:17PM,253230,10,ISDIN Corporation,10,,
4,Dec 15 2022 4:17PM,253232,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 4:39PM,253238,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Dec 15 2022 4:36PM,253236,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Dec 15 2022 4:34PM,253235,19,"GUSA Granules USA, Inc.",2.0,NaN,NaN
3,Dec 15 2022 4:17PM,253230,10,ISDIN Corporation,10.0,NaN,NaN
4,Dec 15 2022 4:17PM,253232,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 4:39PM,253238,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Dec 15 2022 4:36PM,253236,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Dec 15 2022 4:34PM,253235,19,"GUSA Granules USA, Inc.",2.0,0.0,0.0
3,Dec 15 2022 4:17PM,253230,10,ISDIN Corporation,10.0,0.0,0.0
4,Dec 15 2022 4:17PM,253232,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3291839,47.0,4.0,10.0
12,506407,1.0,1.0,0.0
15,506449,16.0,0.0,0.0
16,506360,2.0,0.0,0.0
18,253227,1.0,0.0,0.0
19,506442,3.0,0.0,0.0
21,506391,1.0,1.0,0.0
22,253232,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3291839,47.0,4.0,10.0
1,12,506407,1.0,1.0,0.0
2,15,506449,16.0,0.0,0.0
3,16,506360,2.0,0.0,0.0
4,18,253227,1.0,0.0,0.0
5,19,506442,3.0,0.0,0.0
6,21,506391,1.0,1.0,0.0
7,22,253232,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,47.0,4.0,10.0
1,12,1.0,1.0,0.0
2,15,16.0,0.0,0.0
3,16,2.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,3.0,0.0,0.0
6,21,1.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,47.0
1,12,Released,1.0
2,15,Released,16.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21,22
Status,,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
Released,47.0,1.0,16.0,2.0,1.0,3.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21,22
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Released,47.0,1.0,16.0,2.0,1.0,3.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21,22
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Released,47.0,1.0,16.0,2.0,1.0,3.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()